In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 10      
N = 512 
tau_sim = 1.0     
DTS = 8     

# Glasma
su_group = 'su3'
# Qs = 1.4 
Qs = 2.0     
ns = 50      
factor = 0.8        
uv = 10.0 

# Wong
quark = 'charm'    
mass = 1.275       

# tau_form = 0
initialization = 'pT'         
ntp = 10**5  

# nevents = 10   
nevents = 10
# npTbins = 100 
# pTmax = 10

# representation = 'quantum fundamental'     
representation = 'fundamental' 
boundary = 'periodic'  

# pTs = np.linspace(0, pTmax, npTbins)
pTs = [0, 0.5, 1, 5]

# Store relevant parameters in a dictionary
p = {
    'QUARK': quark,
    'MASS': mass,   
    'QS': Qs,            
    'NEVENTS': nevents,
    # 'NPTBINS': npTbins,   
    'NTP': ntp,   
    # 'PT': pTmax,   
    'PTBINS': pTs,
    }

In [2]:
# rest of parameters  
g2mu = p["QS"] / factor     
g = np.pi * np.sqrt(1 / np.log(p["QS"] / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g2mu     

tau_form = 1/(2*p["MASS"])*hbarc
tau_sim += tau_form

# Results folder
# folder = 'RAA_' + p["QUARK"] + '_fonll_Qs_' + str(p["QS"]) + '_clcasimir'
folder = 'RAA_' + p["QUARK"] + '_fonll_Qs_' + str(p["QS"]) + '_qmcasimir'
filename = 'test_pTs_10_ev.pickle'

In [3]:
import os
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

# Wong modules
from curraun import wong
wong.BOUNDARY = boundary
wong.WONG_TO_HOST = True
from curraun.wong import init_pos, init_charge, init_mom_toy
from curraun.particles_correlators import TransMom

Using CUDA
Using SU(3)
Using double precision
Using double precision


In [4]:
import pickle
from tqdm import tqdm

current_path = os.getcwd() 
results_folder = 'results'
if not os.path.isdir(results_folder):
    os.makedirs(results_folder)
results_path = current_path + '/' + results_folder + '/'

In [5]:
def simulate(p): 
    os.chdir(results_path)

    output = {}
    output['parameters'] = p.copy()
    output['pTs'] = p['PTBINS']
    output['nevents'] = p["NEVENTS"]

    # Derived parameters
    a = L/N
    E0 = N/L * hbarc
    DT = 1.0 / DTS
    formt = int(tau_form / a * DTS)
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    for ipT, pT in enumerate(p['PTBINS']):
        print("pT = " + str(pT) + " GeV")

        output[str(pT)] = {}
        
        for ev in range(0, p["NEVENTS"]):
            # Initialize the Wong solver
            wong_solver = wong.WongSolver(s, ntp)
            x0s, p0s, q0s = np.zeros((ntp, 3)), np.zeros((ntp, 5)), np.zeros((ntp, su.ALGEBRA_ELEMENTS))
            masses = p["MASS"] / E0 * np.ones(ntp)

            for i in range(ntp):
                x0, p0, q0 = init_pos(s.n), init_mom_toy('pT', pT / E0), init_charge(representation)
                x0s[i, :], p0s[i, :], q0s[i, :] = x0, p0, q0
            
            wong_solver.initialize(x0s, p0s, q0s, masses)

            pTs = np.zeros((maxt-formt, ntp))
            compute_pT = TransMom(wong_solver, ntp)

            with tqdm(total=maxt) as pbar:
                for t in range(maxt):
                    # Evolve Glasma fields
                    core.evolve_leapfrog(s)

                    # Solve Wong's equations
                    if t>=formt:  
                        compute_pT.compute()
                        pTs[t-formt] = compute_pT.pT.copy() * E0
                        
                        wong_solver.evolve()

                    pbar.set_description("Event " + str(ev+1))
                    pbar.update(1)

            output[str(pT)]['pTs_event_'+ str(ev+1)] = pTs

    tau = np.linspace(0, tau_sim-tau_form, maxt-formt)
    output['tau'] = tau

    wong_folder = folder
    if not os.path.isdir(wong_folder):
        os.makedirs(wong_folder)
    wong_path = results_path + '/' + wong_folder + '/'
    os.chdir(wong_path)

    with open(filename, 'wb') as handle:
        pickle.dump(output, handle)
    return output

In [6]:
print(p['QUARK'].capitalize() + " quark")

output = simulate(p)

Charm quark
pT = 0 GeV


Event 10: 100%|██████████| 441/441 [01:45<00:00,  4.18it/s]


pT = 0.5 GeV


Event 10: 100%|██████████| 441/441 [01:46<00:00,  4.14it/s]


pT = 1 GeV


Event 10: 100%|██████████| 441/441 [01:55<00:00,  3.83it/s]


pT = 5 GeV


Event 10: 100%|██████████| 441/441 [01:49<00:00,  4.04it/s]


In [14]:
# from old version, should be rewritten
pTs_tau = output['0.5']['pTs_event_1'][1, :]

In [15]:
import matplotlib.pyplot as plt
from scipy import stats

fig, axes = plt.subplots(nrows=1, ncols=1)

pT_bins = np.linspace(np.min(pTs_tau), np.max(pTs_tau), 100)
density = stats.kde.gaussian_kde(pTs_tau)
axes.plot(pT_bins, density(pT_bins))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/n/work00/davrames/condacurraun/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-deb65a2d8b7e>", line 1, in <module>
    import matplotlib.pyplot as plt
  File "/n/work00/davrames/condacurraun/lib/python3.6/site-packages/matplotlib/__init__.py", line 921, in <module>
    dict.update(rcParams, _rc_params_in_file(matplotlib_fname()))
  File "/n/work00/davrames/condacurraun/lib/python3.6/site-packages/matplotlib/__init__.py", line 602, in matplotlib_fname
    for fname in gen_candidates():
  File "/n/work00/davrames/condacurraun/lib/python3.6/site-packages/matplotlib/__init__.py", line 591, in gen_candidates
    yield os.path.join(os.getcwd(), 'matplotlibrc')
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/n/work